In [448]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE






In [449]:
tda = pd.read_csv('C:\\Users\\Eric\\Downloads\\decrypted_file\\train\\train_data_ads.csv')#your directory
tdf =  pd.read_csv('C:\\Users\\Eric\\Downloads\\decrypted_file\\train\\train_data_feeds.csv')

In [450]:
tdf.rename(columns={'u_userId': 'user_id'}, inplace=True)  #change u_userId to user_id for better merging algotithm
tda.rename(columns={'user_id': 'user_id'}, inplace=True)  

tda = tda.sample(frac= 0.03)#remove this row in real process
tdf = tdf.sample(frac= 0.03)#remove this row in real process

In [451]:
#remove time stamp, since we are not interesting about time
tda.drop(columns=['pt_d'], inplace=True)


In [452]:
tda.drop(["device_size"], axis=1, inplace= True) 
tdf.drop(['u_browserMode','e_pl','e_et',"i_s_sourceId", "i_entities"], axis=1, inplace= True)#some duplicated information and not useful information(please double check for me)

In [453]:
df1 = pd.DataFrame(tda)
df2 = pd.DataFrame(tdf)

# Store original data types
original_dtypes_df1 = df1.dtypes
original_dtypes_df2 = df2.dtypes

# merge DataFrames
merged_data = pd.merge(df1, df2, on='user_id', how='outer')

# fill NaN values with appropriate default values
merged_data.fillna(0, inplace=True)

# convert data types back to original
for col in merged_data.columns:
    if col in original_dtypes_df1.index:
        merged_data[col] = merged_data[col].astype(original_dtypes_df1[col])
    elif col in original_dtypes_df2.index:
        merged_data[col] = merged_data[col].astype(original_dtypes_df2[col])

In [454]:
#featuring the data with clicked column
merged_data['clicked'] = merged_data.apply(lambda row: row['user_id'] in tda['user_id'].values and row['user_id'] in tdf['user_id'].values, axis=1)
merged_data.fillna(0, inplace=True )

In [455]:

merged_data.fillna(0, inplace=True )

merged_data.drop(['user_id','log_id','task_id','adv_id','i_docId','device_name','series_dev','slot_id','spread_app_id','site_id','app_score'], axis = 1, inplace= True)
# some data that I would consider to drop based on their distribution. They are either too biased or not meaningful. like ID type of variables. I also delete the column with too many unique value

,label_x,age,gender,residence,city,city_rank,series_group,emui_dev,net_type,creat_type_cd,...,i_dtype,e_ch,e_m,e_po,e_rn,e_section,label_y,cillabel,pro,clicked
0,0.0,5,2,46,354,2,8,32,7,2,...,0,0,0,0,0,0,0.0,0,0,False
1,0.0,5,2,46,354,2,8,32,7,8,...,0,0,0,0,0,0,0.0,0,0,False
2,0.0,5,2,46,354,2,8,32,7,8,...,0,0,0,0,0,0,0.0,0,0,False
3,0.0,5,2,46,354,2,8,32,7,2,...,0,0,0,0,0,0,0.0,0,0,False
4,0.0,3,4,33,319,3,3,20,4,8,...,11,19,1361,12,9,0,-1.0,-1,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743030,0.0,6,4,33,319,3,2,11,7,10,...,0,0,0,0,0,0,0.0,0,0,False
743031,0.0,6,4,33,319,3,2,11,7,5,...,0,0,0,0,0,0,0.0,0,0,False
743032,0.0,6,4,33,319,3,2,11,7,10,...,0,0,0,0,0,0,0.0,0,0,False
743033,0.0,6,4,33,319,3,2,11,7,4,...,0,0,0,0,0,0,0.0,0,0,False


In [459]:
# Split the column into multiple columns(like 23151^231^214^123, I separate this long list to multiple column. I think the numerical number could represent certain category, and after ^ is the subset of this category)

def div_cols(df, col_name):
    split_columns = df[col_name].astype(str).str.split('^', expand=True)
    split_columns.columns = [f'{col_name}_{i+1}' for i in range(split_columns.shape[1])]
    split_columns.fillna(0, inplace=True)

    df = pd.concat([df, split_columns], axis=1)
    df = df.loc[:, ~df.columns.duplicated()]  # drop duplicated columns
    df.drop(col_name, axis=1, inplace=True)
    return df

# Define the split_columns_with_delimiter function
def split_columns_with_delimiter(df, delimiter='^'):
    columns_to_split = [col for col in df.columns if df[col].astype(str).str.contains(delimiter).any()]
   
    for col in columns_to_split:
        df = div_cols(df, col)
   
    return df
merged_data = split_columns_with_delimiter(merged_data)

In [460]:
merged_data.drop(columns=['creat_type_cd_1','inter_type_cd_1'])

,label_x_1,age_1,gender_1,residence_1,city_1,city_rank_1,series_group_1,emui_dev_1,net_type_1,adv_prim_id_1,...,i_dtype_1,e_ch_1,e_m_1,e_po_1,e_rn_1,e_section_1,label_y_1,cillabel_1,pro_1,clicked_1
0,0.0,5,2,46,354,2,8,32,7,1913,...,0,0,0,0,0,0,0.0,0,0,False
1,0.0,5,2,46,354,2,8,32,7,1036,...,0,0,0,0,0,0,0.0,0,0,False
2,0.0,5,2,46,354,2,8,32,7,1492,...,0,0,0,0,0,0,0.0,0,0,False
3,0.0,5,2,46,354,2,8,32,7,1913,...,0,0,0,0,0,0,0.0,0,0,False
4,0.0,3,4,33,319,3,3,20,4,1316,...,11,19,1361,12,9,0,-1.0,-1,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743030,0.0,6,4,33,319,3,2,11,7,1005,...,0,0,0,0,0,0,0.0,0,0,False
743031,0.0,6,4,33,319,3,2,11,7,1236,...,0,0,0,0,0,0,0.0,0,0,False
743032,0.0,6,4,33,319,3,2,11,7,1486,...,0,0,0,0,0,0,0.0,0,0,False
743033,0.0,6,4,33,319,3,2,11,7,1236,...,0,0,0,0,0,0,0.0,0,0,False


In [461]:
#seperate the dataset to 2 file clicked and not clicked
pos_click = merged_data[merged_data['clicked_1'] == "True"]
neg_click = merged_data[merged_data['clicked_1'] == "False"]
# save to csv files
pos_click.to_csv('pos_click.csv', index=False)
neg_click.to_csv('neg_click.csv', index=False)

In [462]:
pos_click

,label_x_1,age_1,gender_1,residence_1,city_1,city_rank_1,series_group_1,emui_dev_1,net_type_1,creat_type_cd_1,...,i_dtype_1,e_ch_1,e_m_1,e_po_1,e_rn_1,e_section_1,label_y_1,cillabel_1,pro_1,clicked_1
4,0.0,3,4,33,319,3,3,20,4,8,...,11,19,1361,12,9,0,-1.0,-1,0,True
5,0.0,3,4,33,319,3,3,20,4,8,...,12,19,1361,1,8,0,-1.0,-1,0,True
6,0.0,3,4,33,319,3,3,20,4,8,...,11,19,1361,5,9,0,-1.0,-1,0,True
7,0.0,3,4,33,319,3,3,20,4,8,...,10,19,1361,6,1,1,-1.0,-1,0,True
8,0.0,3,4,33,319,3,3,20,4,8,...,13,19,1361,7,4,0,-1.0,-1,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743024,0.0,3,2,41,135,5,7,21,7,10,...,12,11,196,1,1,0,-1.0,-1,0,True
743025,0.0,3,2,41,135,5,7,21,7,8,...,12,11,196,1,1,0,-1.0,-1,0,True
743026,1.0,3,2,41,135,5,7,21,7,3,...,12,11,196,1,1,0,-1.0,-1,0,True
743027,0.0,3,2,41,135,5,7,21,7,10,...,12,11,196,1,1,0,-1.0,-1,0,True


In [463]:
#merged_data.drop(['ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',  'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003'], axis=1, inplace= True)